# ADA / Applied Data Analysis
<h2 style="color:#a8a8a8">Homework 4 - Applied Machine Learning<br>
Aimée Montero, Alfonso Peterssen, Cyriaque Brousse</h2>

## Assignment description
In this homework we will gain experience on Applied Machine Learning, exploring an interesting dataset about soccer players and referees.
Given that the focus of this homework is Machine Learning, I recommend you to first take a look at [this notebook](http://nbviewer.jupyter.org/github/mathewzilla/redcard/blob/master/Crowdstorming_visualisation.ipynb)
containing a solid work in pre-processing + visualization of the given dataset. You are *not* allowed to just copy/paste the pre-processing steps
performed by the notebook authors -- you are still supposed to perform your own data analysis for the homework. Still, I'm confident that consulting first
the work done by expert data analysts will speed up tangibly your effort (i.e., they have already found for you many glitches in the data :)


### Assignment
1. Train a `sklearn.ensemble.RandomForestClassifier` that given a soccer player description outputs his skin color. Show how different parameters 
passed to the Classifier affect the overfitting issue. Perform cross-validation to mitigate the overfitting of your model. Once you assessed your model,
inspect the `feature_importances_` attribute and discuss the obtained results. With different assumptions on the data (e.g., dropping certain features even
before feeding them to the classifier), can you obtain a substantially different `feature_importances_` attribute?

  *BONUS*: plot the learning curves against at least 2 different sets of parameters passed to your Random Forest. To obtain smooth curves, partition
your data in at least 20 folds. Can you find a set of parameters that leads to high bias, and one which does not?

2. Aggregate the referee information grouping by soccer player, and use an unsupervised learning technique to cluster the soccer players in 2 disjoint
clusters. Remove features iteratively, and at each step perform again the clustering and compute the silhouette score -- can you find a configuration of features with high silhouette
score where players with dark and light skin colors belong to different clusters? Discuss the obtained results.

## Data description

From a company for sports statistics, we obtained data and profile photos from all soccer players (N = 2053) playing in the first male divisions of England, Germany, France and Spain in the 2012-2013 season and all referees (N = 3147) that these players played under in their professional career. We created a dataset of player–referee dyads including the number of matches players and referees encountered each other and our dependent variable, the number of red cards given to a player by a particular referee throughout all matches the two encountered each other.
 
Player photos were available from the source for 1586 out of 2053 players. Players’ skin tone was coded by two independent raters blind to the research question who, based on their profile photo, categorized players on a 5-point scale ranging from “very light skin” to “very dark skin” with “neither dark nor light skin” as the center value. 

Additionally, implicit bias scores for each referee country were calculated using a race implicit association test (IAT), with higher values corresponding to faster white | good, black | bad associations. Explicit bias scores for each referee country were calculated using a racial thermometer task, with higher values corresponding to greater feelings of warmth toward whites versus blacks. Both these measures were created by aggregating data from many online users in referee countries taking these tests on [Project Implicit](http://projectimplicit.net).

In all, the dataset has a total of 146028 dyads of players and referees. A detailed description of all variables in the dataset can be seen in the list below.

### Variables:

- *playerShort* - short player ID
- *player* - player name
- *club* - player club
- *leagueCountry* - country of player club (England, Germany, France, and Spain)
- *birthday* - player birthday
- *height* - player height (in cm)
- *weight* - player weight (in kg)
- *position* - detailed player position
- *games* - number of games in the player-referee dyad
- *victories* - victories in the player-referee dyad
- *ties* - ties in the player-referee dyad
- *defeats* - losses in the player-referee dyad
- *goals* - goals scored by a player in the player-referee dyad
- *yellowCards* - number of yellow cards player received from referee
- *yellowReds* - number of yellow-red cards player received from referee
- *redCards* - number of red cards player received from referee
- *photoID* - ID of player photo (if available)
- *rater1* - skin rating of photo by rater 1 (5-point scale ranging from “very light skin” to “very dark skin”)
- *rater2* - skin rating of photo by rater 2 (5-point scale ranging from “very light skin” to “very dark skin”)
- *refNum* - unique referee ID number (referee name removed for anonymizing purposes)
- *refCountry* - unique referee country ID number (country name removed for anonymizing purposes)
- *meanIAT* - mean implicit bias score (using the race IAT) for referee country, higher values correspond to faster white | good, black | bad associations
- *nIAT* - sample size for race IAT in that particular country
- *seIAT* - standard error for mean estimate of race IAT
- *meanExp* - mean explicit bias score (using a racial thermometer task) for referee country, higher values correspond to greater feelings of warmth toward whites versus blacks
- *nExp* - sample size for explicit bias in that particular country
- *seExp* - standard error for mean estimate of explicit bias measure

## Part 1 - Familiarizing ourselves with the data

Let's import the required libraries:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
%matplotlib inline

### Importing and cleaning the data

And import the data from the CSV source:

In [17]:
data = pd.read_csv('CrowdstormingDataJuly1st.csv', parse_dates=['birthday'], infer_datetime_format=True)
data.sample(5)

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
18920,anthony-le-tallec,Anthony Le Tallec,Valenciennes FC,France,1984-10-03,184.0,73.0,Right Winger,6,1,...,0.25,373,7,FRA,0.334684,2882.0,0.000151,0.336101,3011.0,0.000586
55000,jacob-butterfield,Jacob Butterfield,Bolton Wanderers,England,1990-06-10,180.0,77.0,Center Midfielder,7,3,...,NaN,1096,44,ENGL,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037
52437,haris-vuckic,Haris Vučkić,Newcastle United,England,1992-08-21,187.0,80.0,NaN,1,1,...,0.25,1014,44,ENGL,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037
67633,marvin-plattenhardt,Marvin Plattenhardt,1. FC Nürnberg,Germany,1992-01-26,181.0,76.0,Left Fullback,1,1,...,0.25,1435,8,DEU,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225
80544,thomas-kahlenberg,Thomas Kahlenberg,VfL Wolfsburg,Germany,1983-03-20,183.0,72.0,Attacking Midfielder,1,0,...,0.00,1740,76,SVK,0.429987,139.0,0.002622,0.668966,145.0,0.012878


First, we follow the intuition that we need to reconcile the `rater1` and `rater2` columns. Indeed, we want a single value on which we will then evaluate the model against. We use the mean as aggregation function:

In [18]:
data['skin'] = (data.rater1 + data.rater2) / 2
data = data.drop('rater1', axis=1)
data = data.drop('rater2', axis=1)

Let's also drop the `photoID` column, since we don't have access to the photos anyways:

In [19]:
data = data.drop('photoID', axis=1)

We notice that the `refCountry` and `Alpha_3` columns are bijective: they contain the same data.<br>
We will keep only the `refCountry` one, since it's an integer value.

In [20]:
data = data.drop('Alpha_3', axis=1)

We also let go of the player full name, since we already have the `playerShort` field that serves that purpose.

In [23]:
data = data.drop('player', axis=1)

Next, we want to convert the birthday (type `datetime` that scikit-learn does not understand) into the player's age:

In [26]:
import datetime
now = datetime.date.today()

data['age'] = [now.year - b.year for b in data.birthday]
data = data.drop('birthday', axis=1)

In [27]:
data.sample(5)

,playerShort,club,leagueCountry,height,weight,position,games,victories,ties,defeats,...,refNum,refCountry,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,skin,age
59513,emmanuel-adebayor,Tottenham Hotspur,England,190.0,75.0,Center Forward,23,9,7,7,...,1214,44,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037,1.00,32
136650,jonas-hofmann,Borussia Dortmund,Germany,176.0,67.0,Right Midfielder,1,0,1,0,...,2905,8,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225,0.00,24
75657,marcel-halstenberg,Borussia Dortmund,Germany,186.0,82.0,Center Back,1,1,0,0,...,1646,8,0.336628,7749.0,0.000055,0.335967,7974.0,0.000225,0.00,25
66615,isma-lopez,Athletic Bilbao,Spain,180.0,76.0,NaN,1,0,1,0,...,1396,3,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.25,26
35841,ross-turnbull,Chelsea FC,England,193.0,85.0,Goalkeeper,1,0,1,0,...,637,44,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037,0.00,31


Also, we need to drop all `N/A` values:

In [30]:
data = data.dropna()

### Dealing with categorical attributes